In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# Load Kaggle Data as matrix
train_csv_data = pd.read_csv('./titanic_train.csv').values
test_csv_data = pd.read_csv('./titanic_test.csv').values
test_csv_sub = pd.read_csv('./titanic_gender_submission.csv').values

# Check loaded data
print("train_csv_data.shape = ", train_csv_data.shape)
print("test_csv_data.shape = ", test_csv_data.shape)
print("test_csv_sub.shape = ", test_csv_sub.shape)

train_csv_data.shape =  (891, 12)
test_csv_data.shape =  (418, 11)
test_csv_sub.shape =  (418, 2)


In [3]:
# train data 에서 male -> 1, female -> 0
for i in range(len(train_csv_data)):
    
    if train_csv_data[i, 4] == 'male':
        
        train_csv_data[i, 4] = 1
    
    else:
        
        train_csv_data[i, 4] = 0
        
# test data 에서 male -> 1, female -> 0
for i in range(len(test_csv_data)):
    
    if test_csv_data[i, 3] == 'male':
        
        test_csv_data[i, 3] = 1
    
    else:
        
        test_csv_data[i, 3] = 0

In [4]:
# train data 에서 Embarked, Empty -> 0, S -> 1, C -> 2, Q -> 3
for i in range(len(train_csv_data)):
    
    if train_csv_data[i, 11] == 'S':
        
        train_csv_data[i, 11] = 1
        
    elif train_csv_data[i, 11] == 'C':
        
        train_csv_data[i, 11] = 2
        
    elif train_csv_data[i, 11] == 'Q':
        
        train_csv_data[i, 11] = 3
        
        
    if np.isnan(train_csv_data[i, 11]):
        
        train_csv_data[i, 11] = 0
        

# test data 에서 Embarked, Empty -> 0, S -> 1, C -> 2, Q -> 3
for i in range(len(test_csv_data)):
    
    if test_csv_data[i, 10] == 'S':
        
        test_csv_data[i, 10] = 1
        
    elif test_csv_data[i, 10] == 'C':
        
        test_csv_data[i, 10] = 2
        
    elif test_csv_data[i, 10] == 'Q':
        
        test_csv_data[i, 10] = 3
        
    if np.isnan(test_csv_data[i, 10]):
        
        test_csv_data[i, 10] = 0

In [5]:
training_x_data = train_csv_data[ : , [ 2, 4, 6, 7, 11 ] ]  # Pclass, Sex, SibSp, Parch, Embarked

training_t_data = train_csv_data[ :, 1:2]

test_x_data = test_csv_data[ : , [ 1, 3, 5, 6, 10 ] ]  # Pclass, Sex, SibSp, Parch, Embarked

test_t_data = test_csv_sub[ :, 1:2]  # test_csv_sub 에서 데이터 가져옴. 주의 요함

In [6]:
print("training_x_data.shape = ", training_x_data.shape, ", test_x_data.shape = ", test_x_data.shape)
print("training_t_data.shape = ", training_t_data.shape, ", test_t_data.shape = ", test_t_data.shape)

training_x_data.shape =  (891, 5) , test_x_data.shape =  (418, 5)
training_t_data.shape =  (891, 1) , test_t_data.shape =  (418, 1)


In [7]:
# Placeholder
X = tf.placeholder(tf.float32, shape=[None, 5])
T = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([5,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

In [8]:
z = tf.matmul(X,W) + b

y = tf.sigmoid(z)

# loss
loss = -tf.reduce_mean(T * tf.log(y) + (1-T) * tf.log(1-y))

In [9]:
learning_rate = 0.01

#Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train = optimizer.minimize(loss)

# Accuracy computation
# True if y > 0.5 else False
predicted = tf.cast(y > 0.5, dtype=tf.float32)

accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, T), dtype=tf.float32))

In [10]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    for step in range(6000):
        
        loss_val, y_val, _ = sess.run([loss, y, train], feed_dict = {X:training_x_data, T:training_t_data})
    
        if step % 500 == 0:
            print("Step =", step, ", loss: ", loss_val)
           
    y_val, predicted_val, accuracy_val = sess.run([y, predicted, accuracy], feed_dict = {X:test_x_data, T:test_t_data})
    
    print("")
    print("Accuracy: " , accuracy_val)

Step = 0 , loss:  0.88840836
Step = 500 , loss:  0.5446618
Step = 1000 , loss:  0.50430286
Step = 1500 , loss:  0.48645929
Step = 2000 , loss:  0.47803855
Step = 2500 , loss:  0.47365135
Step = 3000 , loss:  0.47106978
Step = 3500 , loss:  0.46934843
Step = 4000 , loss:  0.4680697
Step = 4500 , loss:  0.4670431
Step = 5000 , loss:  0.4661764
Step = 5500 , loss:  0.4654219

Accuracy:  0.9904306


In [6]:
print(train_csv_data[0])

[1 0 3 'Braund, Mr. Owen Harris' 'male' 22.0 1 0 'A/5 21171' 7.25 nan 'S']


In [7]:
print(train_csv_data[0, 3])

Braund, Mr. Owen Harris


In [8]:
train_pd_csv = pd.read_csv('./titanic_train.csv')

In [9]:
train_pd_csv.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [10]:
train_pd_csv.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [11]:
train_pd_csv.index

RangeIndex(start=0, stop=891, step=1)

In [12]:
train_pd_csv.values

array([[1, 0, 3, ..., 7.25, nan, 'S'],
       [2, 1, 1, ..., 71.2833, 'C85', 'C'],
       [3, 1, 3, ..., 7.925, nan, 'S'],
       ...,
       [889, 0, 3, ..., 23.45, nan, 'S'],
       [890, 1, 1, ..., 30.0, 'C148', 'C'],
       [891, 0, 3, ..., 7.75, nan, 'Q']], dtype=object)